In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
import joblib
from scipy import sparse
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn import model_selection
from sklearn.model_selection import cross_val_score,cross_validate, KFold
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_curve, auc, classification_report,accuracy_score,precision_score,f1_score,recall_score,cohen_kappa_score,multilabel_confusion_matrix,make_scorer

In [2]:
# load data 
dataframe = pd.read_csv(r'C:\Users\dawns\Desktop\Master thesis\data\Features_level.csv')
#data=dataframe[['CONTENT_NEW','','SCORE']].copy()
df_data=dataframe[['CONTENT_NEW','WORDCUT','SCORE','LEVEL','Tokens','level1_tokens','level2_tokens','level3_tokens','level4_tokens','level5_tokens','level6_tokens','level7_tokens','char_num','high_level','primary_level','Connective_ratio','connective_SCORE','pre','connective_numbers']].copy()
df_data.head()

# Generating 'y' for classification as well as regression
y = df_data['SCORE']
#y

# generating x 
pre = df_data['pre']


In [3]:
#ngrams Vectorizer
print(pre.shape)
cv = CountVectorizer(ngram_range=(1, 3), max_features=1000)
ngram_vector = cv.fit_transform(pre)
print(ngram_vector.shape)




(3668,)
(3668, 1000)


In [4]:
#保存ngrams特征 save ngrams features
print(type(ngram_vector))

# save your model in disk
joblib.dump(ngram_vector, 'ngram_vector.pkl') 

# load your model
ngram_vector = joblib.load('ngram_vector.pkl') 

<class 'scipy.sparse._csr.csr_matrix'>


In [5]:
# 特征合并 feature concatenate 
feature_set1 = df_data[['connective_SCORE',]]
feature_set2 = ngram_vector
print(feature_set1.shape)
print(feature_set2.shape)
train_features = sparse.hstack((feature_set2, feature_set1))
print('=============')
print(train_features.shape)


(3668, 1)
(3668, 1000)
(3668, 1001)


In [6]:
# 划分训练集 ，测试集 train_test split
x_train, x_test, y_train, y_test = train_test_split(train_features, y, test_size=0.2)
print(x_train.shape)
print(y_train.shape)


(2934, 1001)
(2934,)


In [7]:
# linear regression 线性回归
from sklearn.linear_model import LinearRegression
linear_regressor = LinearRegression()
linear_regressor.fit(x_train, y_train)
y_pred = linear_regressor.predict(x_test)
r2 = r2_score(y_test,y_pred)
neg_mean_absolute_error = mean_absolute_error(y_test,y_pred)
neg_mean_squared_error = mean_squared_error(y_test,y_pred)
print('neg_mean_absolute_error:',neg_mean_absolute_error)
print('neg_mean_squared_error:',neg_mean_squared_error)
print('r2:',r2)

neg_mean_absolute_error: 8.003773964009833
neg_mean_squared_error: 104.232860841904
r2: 0.1782694578227767


In [8]:
df= pd.DataFrame(y_pred)
print(df.min())
print(df.max())

0    48.210641
dtype: float64
0    119.867285
dtype: float64


In [9]:
df= pd.DataFrame(y_pred)
print(df.min())
print(df.max())
df[(df >= 95)]=95
df[(df >= 90) & (df <95)]=90
df[(df >= 85) & (df < 90)]=85
df[(df >= 80) & (df < 85)]=80
df[(df >= 75) & (df < 80)]=75
df[(df >= 70) & (df < 75)]=70
df[(df >= 65) & (df < 70)]=65
df[(df >= 60) & (df < 65)]=60
df[(df >= 55) & (df < 60)]=55
df[(df >= 50) & (df < 55)]=50
df[(df >= 45) & (df < 50)]=45
df[ (df < 45)]=40


0    48.210641
dtype: float64
0    119.867285
dtype: float64


In [10]:
y_pred

array([60., 80., 75., 70., 70., 65., 60., 70., 70., 85., 65., 60., 60.,
       60., 70., 70., 75., 70., 65., 80., 65., 85., 90., 75., 70., 70.,
       65., 70., 90., 70., 75., 60., 75., 65., 55., 75., 60., 65., 70.,
       65., 80., 85., 70., 80., 75., 60., 55., 60., 65., 55., 80., 70.,
       60., 85., 65., 80., 85., 90., 80., 55., 50., 55., 75., 90., 95.,
       60., 60., 90., 70., 75., 65., 70., 65., 70., 70., 55., 60., 70.,
       75., 75., 55., 70., 60., 60., 80., 75., 55., 85., 70., 70., 60.,
       60., 80., 65., 90., 65., 65., 60., 70., 70., 60., 70., 75., 75.,
       80., 60., 70., 60., 60., 70., 60., 75., 70., 55., 50., 60., 50.,
       65., 60., 60., 75., 60., 70., 60., 50., 70., 60., 55., 75., 65.,
       70., 75., 70., 75., 85., 75., 50., 60., 65., 75., 65., 65., 65.,
       65., 70., 70., 65., 70., 50., 65., 75., 55., 60., 65., 60., 85.,
       55., 70., 50., 70., 65., 70., 75., 70., 55., 60., 75., 70., 65.,
       80., 60., 80., 55., 80., 65., 75., 65., 60., 65., 80., 75

In [11]:
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)

qwkappa:
 0.521325957952848


In [42]:
# 交叉验证 cross validation linear
cv_scores =[]
scores = cross_val_score(linear_regressor,x_train,y_train,cv=10,scoring='r2')
cv_scores.append(scores)
print(cv_scores)
print(scores.mean())


[array([ 0.14358933,  0.1093418 ,  0.20649301,  0.10630298,  0.2310016 ,
        0.0067015 ,  0.13485366, -0.02066288,  0.20402745,  0.09947789])]
0.12211263463382278


In [12]:

# 随机森林 random forest

rf = ensemble.RandomForestRegressor()
params = {'n_estimators': [100], 'max_depth':[100], 'max_features':[10]}
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print('r2:',r2_score(y_test,y_pred))
print('mean_squared_error:',mean_squared_error(y_test,y_pred))
print('mean_absolute_error:',mean_absolute_error(y_test,y_pred))


r2: 0.34573556017957596
mean_squared_error: 82.99053133514985
mean_absolute_error: 7.071798365122614


In [13]:
df= pd.DataFrame(y_pred)
print(df.min())
print(df.max())
df[(df >= 95)]=95
df[(df >= 90) & (df <95)]=90
df[(df >= 85) & (df < 90)]=85
df[(df >= 80) & (df < 85)]=80
df[(df >= 75) & (df < 80)]=75
df[(df >= 70) & (df < 75)]=70
df[(df >= 65) & (df < 70)]=65
df[(df >= 60) & (df < 65)]=60
df[(df >= 55) & (df < 60)]=55
df[(df >= 50) & (df < 55)]=50
df[(df >= 45) & (df < 50)]=45
df[ (df < 45)]=40
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)


0    50.1
dtype: float64
0    88.15
dtype: float64
qwkappa:
 0.4762137775850117


In [43]:
# randomforest regressor
cv_scores =[]
scores = cross_val_score(rf,x_train,y_train,cv=10,scoring='r2')
cv_scores.append(scores)
print(cv_scores)
print(scores.mean())

[array([0.34288368, 0.37797672, 0.33201683, 0.32030635, 0.39741044,
       0.30673296, 0.37009986, 0.359065  , 0.39029593, 0.32710491])]
0.3523892687262017


In [14]:
#xgboost  xgboost处理回归
model = xgb.XGBRegressor()
params = {'n_estimators': [160], 'learning_rate': [0.1]}
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print('R2:',r2_score(y_test,y_pred))
print('mean_squared_error:',mean_squared_error(y_test,y_pred))
print('mean_absolute_error:',mean_absolute_error(y_test,y_pred))


R2: 0.3180124044019982
mean_squared_error: 86.50709021904672
mean_absolute_error: 7.203212291083479


In [15]:
df= pd.DataFrame(y_pred)
print(df.min())
print(df.max())
df[(df >= 95)]=95
df[(df >= 90) & (df <95)]=90
df[(df >= 85) & (df < 90)]=85
df[(df >= 80) & (df < 85)]=80
df[(df >= 75) & (df < 80)]=75
df[(df >= 70) & (df < 75)]=70
df[(df >= 65) & (df < 70)]=65
df[(df >= 60) & (df < 65)]=60
df[(df >= 55) & (df < 60)]=55
df[(df >= 50) & (df < 55)]=50
df[(df >= 45) & (df < 50)]=45
df[ (df < 45)]=40
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)

0    42.975655
dtype: float32
0    93.980515
dtype: float32
qwkappa:
 0.5043740870850772


In [18]:
print(y_pred.shape)
print(type(y_pred))
print(y_test.shape)
print(y_test)

(734,)
<class 'numpy.ndarray'>
(734,)
3661    65
578     90
629     85
473     90
857     70
        ..
1670    70
2023    60
1676    75
89      85
2963    70
Name: SCORE, Length: 734, dtype: int64


In [44]:
#xgboost
cv_scores =[]
scores = cross_val_score(model,x_train,y_train,cv=10,scoring='r2')
cv_scores.append(scores)
print(cv_scores)
print(scores.mean())

[array([0.334737  , 0.37429865, 0.28518211, 0.26621921, 0.36092308,
       0.29057575, 0.34972406, 0.29749909, 0.338318  , 0.26185855])]
0.3159335493712757


In [44]:

df= pd.DataFrame(y_pred)
df


,0,result
0,71.429034,71.429034
1,71.372675,71.372675
2,65.495903,65.495903
3,83.370579,83.370579
4,82.374619,82.374619
...,...,...
729,65.205320,65.205320
730,60.930959,60.930959
731,65.985023,65.985023
732,79.202493,79.202493
